In [ ]:
# Mount drive on google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install libraries and dependencies
!pip install python-dotenv
!pip install openai
!pip install pypdf
!pip install -U langchain-community
!pip install -U langchain-openai
!pip install langchain<0.3.0,>=0.2.13
!pip install chromadb>=0.5.3
!pip install tiktoken
!pip install langsmith<0.2.0,>=0.1.75

/bin/bash: line 1: 0.3.0,: No such file or directory
/bin/bash: line 1: 0.2.0,: No such file or directory


In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

_ = load_dotenv(find_dotenv()) # read local .env file

os.environ['OPENAI_API_KEY'] = "ENTER-YOUR-OPENAI-API-KEY-HERE"

In [ ]:

# Load the file from the drive
loader = PyPDFLoader("./sample_data/mental_health_in_uk.pdf")
pages = loader.load()

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_API_KEY"] = "ENTER-YOUR-LANGCHAIN-API-KEY-HERE"

: 

In [ ]:
# Directory to persist vector embeddings and data to
persist_directory = './sample_data/TEST/'

# Create embeddings object
embedding = OpenAIEmbeddings()

: 

In [ ]:
# Split documents into chunks and overlap for context
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(pages)

: 

In [ ]:
# Create vector database
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)
vectordb

In [ ]:
# Check the vector db works by quering it using a similarity search
question = "Which age range exhibits the highest likelihood of mental health issues ?"
docs = vectordb.similarity_search(question,k=3)
docs

: 

In [ ]:
# Create a memory buffer to store conversation history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

: 

In [ ]:
# Create vector db retriever
retriever=vectordb.as_retriever()

# Use conversational retrieval chain for interacting with the llm
qa = ConversationalRetrievalChain.from_llm(
   llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    memory=memory
)

: 

In [ ]:
# First question
question = "Which age range exhibits the highest likelihood of mental health issues?"
result = qa({"question": question})
result['answer']

: 

In [ ]:
# Second question
question = "What can be done to help them?"
result = qa({"question": question})
result['answer']

: 

: 